# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

## 2. Load the simulation data

In [3]:

# Set the working directory
indir = "/Users/an/Documents/0_mfpt/repptis1/"  

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_2108"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_repptismazegap2708"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/i_star/simulations/RETIS_flat_br_noswap"
indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS"

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS
['/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/000', '/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/001', '/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/002', '/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/003', '/run/user/1001/gvfs/smb-share:

In [4]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

    
    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    #### CHANGE HERE ####
    # pe.set_orders(load=False, acc_only=True, save=True)        # for the 1st time you run this notebook for a certain simulation, this will store .npy files
    pe.set_orders(load=True, acc_only=True, save=False)                  # for the next times, you can read npy files (save=True/False is not important)
    # pe.set_orders(load=False, acc_only=True, save=False)     # if saving doesn't work

################################################################################
/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/000
pathensemble info: 
{'cyclenumbers': array([    0,     1,     2, ..., 29998, 29999, 30000]),
 'dirs': array([ 0.,  1.,  0., ..., 35.,  0., 30.]),
 'flags': array(['ACC', 'ACC', 'BTL', ..., 'ACC', 'BTL', 'ACC'], dtype='<U3'),
 'generation': array(['ki', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[-0.1, -0.1, -0.1], ['l_[0]', 'l_[0]', 'l_[0]']],
 'istar_idx': array([[ 0,  0],
       [ 0,  1],
       [ 0,  1],
       ...,
       [ 0, 50],
       [ 0, 29],
       [ 0, 14]]),
 'lambmaxs': array([-0.08922184, -0.08137773, -0.10142089, ..., -0.09693182,
       -0.11363313, -0.09685035]),
 'lambmins': array([-0.105     , -0.10

## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

In [5]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        data[i] = {}
        continue  #  [0-] is not used for Pcross calculations

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 2859
wLMR = 2631
wLML = 24511
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.09693464004126447
pLML = 0.9030653599587355
Local crossing probabilities:
p2R = 0.08769707676410786
p2L = 0.9123029232358921
doing pathensemble 2
Weights of the different paths:
wRMR = 6996
wRML = 8055
wLMR = 7743
wLML = 7207
Local crossing probabilities:
pRMR = 0.46481961331472993
pRML = 0.5351803866852701
pLMR = 0.5179264214046823
pLML = 0.48207357859531774
Local crossing probabilities:
p2R = 0.49128362387920405
p2L = 0.508716376120796
doing pathensemble 3
Weights of the different paths:
wRMR = 7208
wRML = 7952
wLMR = 7833
wLML = 7008
Local crossing probabilities:
pRMR = 0.47546174142480213
pRML = 0.5245382585751979
pLMR = 0.5277946230038407
pLML = 0.4722053769961593
Local crossing probabilities:
p2R = 0.5013499550014999
p2L = 0.4986500449985001
doing pathensemble 4
Weights of the different paths:
wRMR = 7

### Generate pathlength distribution figures, as in PyRETIS reports.

In [6]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


### Compute Pcross using in-house functions

In [7]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [8]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar([i for i in range(len(Pcrossfull))], Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 0.1844769368893792
which is the case!
[1.0, 0.5179264214046823, 0.2733587803290126, 0.1844769368893792]
[0.5179264214046823, 0.5277946230038407, 0.6748527948044841]
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [9]:
# TODO DONT INCLUDE??

# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"]["LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])

This should be the same as the repptis_report.pdf value: 0.025518974278035258


## 4. Analysis using the MSM

### Construct transition matrix M

In [10]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

[-0.1, 0.0, 0.1, 0.2, 0.3]
N 5
NS 15


In [11]:
print("mm", pmms)
print("mp", pmps)
print("pm", ppms)
print("pp", ppps)
print("sum", np.array(pmms)+np.array(pmps))
print("sum", np.array(ppms)+np.array(ppps))
if N > 3:  
    M = construct_M(pmms, pmps, ppms, ppps, N)
elif N == 3:
    M = construct_M_N3(pmms, pmps, ppms, ppps, N)
else:
    raise ValueError("The amount of interfaces needs to be 3 at least!")

mm [0.9030653599587355, 0.48207357859531774, 0.4722053769961593, 0.4787678619574009]
mp [0.09693464004126447, 0.5179264214046823, 0.5277946230038407, 0.521232138042599]
pm [1.0, 0.5351803866852701, 0.5245382585751979, 0.5305637982195845]
pp [0.0, 0.46481961331472993, 0.47546174142480213, 0.46943620178041545]
sum [1. 1. 1. 1.]
sum [1. 1. 1. 1.]


In [12]:
# We can print the transition matrix M and check that all rows sum to 1.
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)

M
shape (15, 15)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.90306536 0.09693464 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.48207358 0.51792642
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.47220538 0.52779462 0.         0.
  0.       

### Look at this Markov model
*INCLUDE?*

In [13]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)
pprint(M)

[-1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
 -6.02003162e-01+0.00000000e+00j  6.02003162e-01+0.00000000e+00j
 -1.17138467e-01+0.00000000e+00j  5.45081240e-16+1.61051996e-01j
  5.45081240e-16-1.61051996e-01j  1.17138467e-01+0.00000000e+00j
  1.21540231e-08+0.00000000e+00j -1.21540255e-08+0.00000000e+00j
  1.05194348e-15+0.00000000e+00j -1.37558135e-17+5.45992998e-17j
 -1.37558135e-17-5.45992998e-17j  8.33616087e-19+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
[ 1.00000000e+00+0.j       -1.00000000e+00+0.j
  6.02003162e-01+0.j       -6.02003162e-01+0.j
  1.05786578e-16+0.161052j  1.05786578e-16-0.161052j
 -1.17138467e-01+0.j        1.17138467e-01+0.j
 -2.97931323e-09+0.j        2.97931088e-09+0.j
  6.40803555e-16+0.j        2.25906124e-15+0.j
 -1.41795489e-16+0.j        4.15704427e-17+0.j
  2.51685088e-17+0.j      ]
array([[0.        , 0.90306536, 0.09693464, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.  

In [14]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[0,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.90306536 0.         0.         0.         0.04672963 0.05020501
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
1.0
[0.         0.81552704 0.08753832 0.04672963 0.         0.
 0.         0.         0.02370708 0.02649793 0.         0.
 0.         0.         0.        ]
0.9999999999999999
[0.86225667 0.         0.         0.         0.04219991 0.04533841
 0.01268756 0.01101951 0.         0.         0.         0.
 0.01268636 0.01381158 0.        ]
0.9999999999999999
[0.         0.77867413 0.08358254 0.05488747 0.         0.
 0.         0.         0.02661251 0.02974541 0.00665448 0.00603188
 0.         0.         0.01381158]
0.9999999999999999
[0.84737318 0.         0.         0.         0.04029293 0.04328961
 0.01780384 0.01546315 0.         0.         0.         0.
 0.01712901 0.01864827 0.        ]
0.9999999999999998
[0.         0.76523337 0.08213981 0.05809678 0.         0.
 0.         0.         0.0277433

### Pcross with MSM

In [15]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

Z
state 0-     : 0
state B      : 1
state 0+- LML: 0.0
state 0+- LMR: 0.2632595970560368
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.5082953604530221
state 1+- RML: 0.0
state 1+- RMR: 0.5082953604530221
state 2+- LML: 0.23626565289544504
state 2+- LMR: 0.7516739115271318
state 2+- RML: 0.23626565289544504
state 2+- RMR: 0.7516739115271318
state 3+- LML: 0.48132256108919164
state 3+- LMR: 1.0
Y
state 0-     : 0.025518974278035254
state B      : 0.0
state 0+- LML: 0.0
state 0+- LMR: 0.2632595970560368
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.5082953604530221
state 1+- RML: 0.0
state 1+- RMR: 0.5082953604530221
state 2+- LML: 0.23626565289544502
state 2+- LMR: 0.7516739115271318
state 2+- RML: 0.23626565289544502
state 2+- RMR: 0.7516739115271318
state 3+- LML: 0.48132256108919164
state 3+- LMR: 1.0

Global crossing probability:  0.025518974278035254


### Pathlength analysis

In [16]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

0
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
1
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
2
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
3
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
4
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.


In [17]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

5/30k-cycles/REPPTIS
AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE

Ensemble 0 (/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/000):
--------------------------------------------------
  All accepted paths:    66.39 (n=12200, weighted)
------------------------------
  LML :     0.00 (n=   0, weighted)
  LMR :     0.00 (n=   0, weighted)
  RMR :    66.39 (n=12200, weighted)
  RML :     0.00 (n=   0, weighted)
  LM* :     0.00 (n=   0, weighted)
  *M* :     0.00 (n=   0, weighted)
  *** :     0.00 (n=   0, weighted)
  RM* :     0.00 (n=   0, weighted)
  L** :     0.00 (n=   0, weighted)
  **R :     0.00 (n=   0, weighted)
  R** :     0.00 (n=   0, weighted)

Ensemble 1 (/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/RE

In [18]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

Collect tau
ensemble 0 /run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/000
ensemble 1 /run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/001
ensemble 2 /run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/002
ensemble 3 /run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS/003
ensemble 4 /run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gam

In [19]:
# Look at computed taus
print("tau")
print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
print("\ntau1")
print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
print("\ntaum")
print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
print("\ntau2")
print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)

tau
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan         64.4
1     001            9.2         50.3         48.8          nan
2     002          192.0        185.8        179.3        192.5
3     003          192.7        185.3        182.6        191.2
4     004          189.0        182.0        178.8        190.9

tau1
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan          0.0
1     001            0.0          0.0         48.8          nan
2     002           50.4         49.7         48.9         50.0
3     003           50.0         49.6         49.0         50.5
4     004           50.0         49.6         49.8         50.0

taum
Index Name            mm           mp           pm           pp
--------------------------------------------

In [20]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
tau_m = tau-tau1-tau2  # yes, this is the same thing as taum

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

tau
[ 64.38558715   9.22638815  50.26795895  48.7838405  192.03760233
 185.77205218 179.30651769 192.49442539 192.69634703 185.34239755
 182.57683602 191.18007769 189.02590455 181.97555929   0.        ]


tau1
[ 0.          0.          0.         48.7838405  50.3790759  49.66692496
 48.91731844 49.99957118 50.00028539 49.59428061 48.98503521 50.52261376
 49.98761087 49.60080176  0.        ]
taum
[6.43855871e+01 9.22638815e+00 7.60167237e-04 0.00000000e+00
 9.10649369e+01 8.60402945e+01 7.97143389e+01 9.26112064e+01
 9.32424372e+01 8.43125239e+01 8.35983400e+01 9.12101831e+01
 8.87488385e+01 8.20037502e+01 0.00000000e+00]
tau2
[ 0.          0.         50.26719878  0.         50.59358957 50.06483275
 50.67486034 49.8836478  49.45362443 51.435593   49.99346076 49.4472808
 50.28945516 50.37100737  0.        ]


tau = tau1+taum+tau2 => difference is 3.0292258760486853e-28


## 5. Flux calculation

### Collect tau for [0+]

In [21]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

G
state 0-     : 0.0
state B      : 0.0
state 0+- LML: 9.22638815225817
state 0+- LMR: 458.1183359778656
state 0+- RML: 0.0
state 1+- LML: 141.65852643263491
state 1+- LMR: 655.615412185471
state 1+- RML: 130.38919925512104
state 1+- RMR: 662.0051391761973
state 2+- LML: 520.1907765250181
state 2+- LMR: 518.9014652634722
state 2+- RML: 511.0865156860114
state 2+- RMR: 523.8108122512936
state 3+- LML: 656.1747255680734
state 3+- LMR: 132.37475753265227
H
state 0-     : 52.739567532155505
state B      : 0.0
state 0+- LML: 0.0
state 0+- LMR: 458.1175758106288
state 0+- RML: 0.0
state 1+- LML: 50.593589565700015
state 1+- LMR: 569.5751177259591
state 1+- RML: 50.67486033519553
state 1+- RMR: 569.393932772538
state 2+- LML: 426.94833931040614
state 2+- LMR: 434.58894132628336
state 2+- RML: 427.4881756457699
state 2+- RMR: 432.6006291214378
state 3+- LML: 567.4258870491377
state 3+- LMR: 50.37100737100738

tau [0+]:  [52.73956753]


### The flux

In [22]:
flux = 1/(tau[0]+h1[0][0])
dt = 0.0002 # Change if needed
flux
print(flux/dt, "1/time")

42.68937798795308 1/time


## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [26]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0]/dt)

The rate constant k is:  0.9375893276918338


## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [28]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False) #, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

G
state B: 0.0
state 0-     : 5332.82520643558
state 0+- LML: 5332.82520643558
state 0+- LMR: 2495.432199302723
state 0+- RML: 5332.831589838732
state 1+- LML: 5332.831589838732
state 1+- LMR: 1876.5116619773203
state 1+- RML: 5964.957933027679
state 1+- RMR: 1253.302992958078
state 2+- LML: 5964.957933027679
state 2+- LMR: 1253.302992958078
state 2+- RML: 6599.847546473057
state 2+- RMR: 633.4510064438982
state 3+- LML: 6599.847546473057
state 3+- LMR: 633.4510064438982
H
state B: 5332.82520643558
state 0-     : 5332.82520643558
state 0+- LML: 5332.82520643558
state 0+- LMR: 2495.432199302723
state 0+- RML: 5332.831589838732
state 1+- LML: 5332.831589838732
state 1+- LMR: 1876.5116619773203
state 1+- RML: 5964.957933027679
state 1+- RMR: 1253.302992958078
state 2+- LML: 5964.957933027679
state 2+- LMR: 1253.302992958078
state 2+- RML: 6599.847546473057
state 2+- RMR: 633.4510064438982
state 3+- LML: 6599.847546473057
state 3+- LMR: 633.4510064438982
interesting
[5332.82520644]


In [32]:
k_flux_pcross = flux * y1[0][0] / dt
k_mfpt = 1 / (mfpt * dt)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")


Rate constant from P_cross × flux: 9.3758932769e-01 [1/ps]
Rate constant from MFPT:           9.3758932769e-01 [1/ps]
Relative difference:               0.00%
